### Installing Dependencies

In [5]:
%pip install langchain
%pip install langchain-ollama
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#pip install -U langchain langchain-openai

### Load ENV

In [6]:
from dotenv import load_dotenv
import os
load = load_dotenv('./../.env')

print(os.getenv("LANGSMITH_API_KEY"))

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


### Interacting with LLM

In [7]:
from langchain_ollama import ChatOllama

llm =  ChatOllama(
    base_url = "http://localhost:11434",
    model = "deepseek-r1:8b",
    temperature= 0.5 ,
    max_tokens = 1000
)

response = llm.invoke("hello, How are you doing?")

print(response)

content='<think>\nOkay, so I just got a message from someone saying "hello, How are you doing?" and then the assistant responded with a greeting. I\'m trying to figure out how to respond properly. Let me think through this step by step.\n\nFirst, when someone greets me with a simple "hello," it\'s usually polite to return the greeting. So I should probably say something like "Hello!" That seems standard.\n\nNext, they\'re asking "How are you doing?" That\'s a common question used to inquire about someone\'s well-being or current state. The assistant responded by saying it was doing well and asked how I am. So I should follow that pattern.\n\nI might want to add a bit more to make the conversation flow naturally. Maybe respond with how I am doing, then ask a question in return. That way, the conversation continues. For example: "Hello! I\'m doing well, thank you. How can I assist you today?"\n\nWait, should I include an exclamation mark? The assistant used one, so maybe that\'s appropri

#### Structured output

In [3]:
import re
import json
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage # Import AIMessage for type hinting (optional but good practice)

# --- Initialize the LLM (same as before) ---
llm =  ChatOllama(
    base_url = "http://localhost:11434",
    model = "deepseek-r1:8b",
    temperature= 0.5
)

# --- Invoke the LLM (same as before) ---
prompt = "What are AI agents? What is the difference between AI, AI agents, AI workflow, automation etc"
print(f"Sending prompt: \"{prompt}\"\n")
response: AIMessage = llm.invoke(prompt) # Added type hint


# --- Structure and Visualize the Output ---

# 1. Extract the full content string
full_content = response.content

# 2. Separate the <think> block and the main response using regex
think_content = "N/A" # Default if not found
main_response = full_content # Default if no think block

# Use regex to find the think block (dot matches newline)
# Assuming the model *might* output <think> tags based on the original code's intent
# If your model doesn't use <think>, this part might not find anything, which is fine.
match = re.search(r"<think>(.*?)</think>", full_content, re.DOTALL)
if match:
    think_content = match.group(1).strip() # Get content inside tags
    # Find the end of the </think> tag to get the main response after it
    end_think_tag_index = match.end()
    main_response = full_content[end_think_tag_index:].strip()
else:
    # If no <think> block, assume the whole content is the main response
    # and the thinking process is not explicitly separated in the output.
    main_response = full_content.strip()
    think_content = "(No <think> block found in response)"


# 3. Extract Metadata
response_meta = response.response_metadata
usage_meta = response.usage_metadata
other_info = {
    "id": response.id,
    "additional_kwargs": response.additional_kwargs,
    # Add type if needed, though it's usually clear from context
    "type": response.type
}

# --- FIX: Define a default serializer function for json.dumps ---
def default_serializer(obj):
    """Convert non-serializable objects to strings."""
    try:
        # Let the default encoder try first
        return json.JSONEncoder().default(obj)
    except TypeError:
        # If it fails, convert the object to its string representation
        return str(obj)

# 4. Print the structured output
print("="*50)
print("--- Thinking Process ---")
print("="*50)
print(think_content)
print("\n") # Add a newline for spacing

print("="*50)
print("--- Main Response ---")
print("="*50)
print(main_response)
print("\n")

print("="*50)
print("--- Response Metadata ---")
print("="*50)

# --- FIX: Use the default serializer in json.dumps ---
print(json.dumps(response_meta, indent=4, default=default_serializer))
print("\n")

print("="*50)
print("--- Usage Metadata ---")
print("="*50)
# Assuming usage_meta is already JSON serializable (usually is)
# If you get an error here too, apply the same default=default_serializer fix
print(json.dumps(usage_meta, indent=4))
print("\n")

print("="*50)
print("--- Other Info ---")
print("="*50)
# other_info should be serializable as constructed
print(json.dumps(other_info, indent=4))
print("="*50)


Sending prompt: "What are AI agents? What is the difference between AI, AI agents, AI workflow, automation etc"

--- Thinking Process ---
Okay, so I'm trying to understand what AI agents are. From what I remember, AI stands for Artificial Intelligence, which is about machines being able to perform tasks that usually require human intelligence. But then there's this term "AI agent," and I'm not entirely sure how it fits into all of this.

I think an AI agent might be some kind of software program, right? Maybe like a robot or something that can make decisions on its own. But wait, isn't AI more about learning and adapting, while agents are more like autonomous entities? Hmm, maybe I should break it down further.

So, if AI is the broader field, then AI agents must be a specific type of system within AI. Maybe they're like the ones that take inputs and process them to make decisions or actions without human intervention. For example, chatbots are AI agents because they interact with user

### Prompt Template

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("What is the advantage of running the LLM in {env}");

prompt = prompt_template.invoke({"env": "local machine"})

print(prompt)

content = llm.invoke(prompt).content



#print(content)

### Chat Prompt Template

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

systemMessage = SystemMessagePromptTemplate.from_template("You are an LLM expert")
humanMessage = HumanMessagePromptTemplate.from_template("What is the advantage of running AI Models in {env}")

prompt_template = ChatPromptTemplate([
    systemMessage, 
    humanMessage
])

# 1 way of doing it
# prompt_template = ChatPromptTemplate([
#     ("system", "You are an LLM expert"),
#     ("user", "What is the advantage of running AI Models in {env}")
# ])

prompt_template

prompt = prompt_template.invoke({"env": "local machine"})

# print(prompt)

content = llm.invoke(prompt).content

print(content)

### Message Placeholder

In [10]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder
)

from langchain_core.messages import HumanMessage

systemMessage = SystemMessagePromptTemplate.from_template("You are an LLM expert")
humanMessage = [HumanMessage("What is the advantage of running LLM in local machine")]

prompt_template = ChatPromptTemplate([
    systemMessage,
    ("placeholder", "{msg}")
])

prompt = prompt_template.invoke({"msg": humanMessage})

content = llm.invoke(prompt).content
print(content)
# for str in llm.stream(prompt):
#      print(str.content)

<think>
Okay, so I'm trying to understand the advantages of running a Large Language Model (LLM) on a local machine. I know that most people use these models online through platforms like ChatGPT or via APIs, but sometimes they might consider running them locally. Let me think about why someone would want to do that.

First, maybe privacy is a concern. If I run the model on my own machine, I don't have to send all my inputs and outputs over the internet. That means whatever I'm working on or discussing remains private. This could be important for handling sensitive information or just keeping my conversations more secure.

Another thought is about control. When you run an LLM locally, you might have more say in how it's configured. You can choose different versions of the model, maybe even modify or fine-tune them according to specific needs. This customization could make the model more effective for particular tasks, like customer service or creative writing.

Cost is another factor. 